In [26]:
import pandas as pd
import sqlite3
import os

from copy import deepcopy

In [42]:
class Database:
    
    def __init__(self, db_path):
        self.path = db_path
        self.con = sqlite3.connect(self.path)
    
    def fetch(self, sql):
        c = self.con.cursor()
        return pd.read_sql(sql, self.con)
    
    def import_csv_files_from_dir(dir_name):
        csv_files = [
            os.path.join(dir_name, foo)
            for foo 
            in os.listdir(dir_name) if foo.endswith(".csv")]

        for csv_file in csv_files:
            db.create_table_by_csv_file(csv_file)
    
    def create_table_by_csv_file(self, file_path):
        table_name = os.path.basename(file_path).split(".")[0]
        df = pd.read_csv(file_path)
        print(f"Creating table: {table_name}")
        return df.to_sql(table_name, self.con, if_exists="replace")
    
    def __del__(self):
        self.con.close()

In [54]:
class QueryManager:
    
    def __init__(self, db):
        self.db = db
        
    def get_airports_by_us_state_and_runway(self, us_state: str, runway: int):
        other_runway = runway - 18 if runway > 18 else runway + 18
        runways = ", ".join([str(runway).zfill(2), str(other_runway).zfill(2)])
        print(f"Filtering by state: {us_state} and runways {runways}")
        
        q = f"""
        WITH rel_airports AS (
            SELECT
                *
            FROM 
                airports
            WHERE 
                iso_region = "{us_state}"
        ),
        rel_runways AS (
            SELECT
                airport_ident
            FROM 
                runways
            WHERE 
                he_ident IN ({runways}) OR le_ident IN ({runways})
        )

        SELECT
            rel_airports.ident,
            rel_airports.latitude_deg,
            rel_airports.longitude_deg
        FROM rel_airports
        JOIN runways
            ON rel_airports.ident = runways.airport_ident
        """
        
        return self.db.fetch(q)

In [55]:
db = Database("database.sqlite")
qm = QueryManager(db)

In [57]:
qm.get_airports_by_us_state_and_runway("US-ID", 19)

Filtering by state: US-ID and runways 19, 01


,ident,latitude_deg,longitude_deg
0,00ID,48.145302,-116.213997
1,00ID,48.145302,-116.213997
2,01ID,42.608200,-112.031998
3,02ID,44.555500,-115.305000
4,03ID,44.794300,-116.532997
...,...,...,...
309,US-0999,45.020639,-116.437389
310,US-1115,42.925555,-115.757222
311,US-1116,44.741389,-114.918472
312,US-1126,42.781421,-112.849347
